In [6]:
# Встановлюємо бібліотеку surprise
!pip install scikit-surprise

# Імпортуємо необхідні бібліотеки
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.model_selection import KFold
from surprise import accuracy
import pandas as pd

# Завантажуємо датасет
data = Dataset.load_builtin('ml-100k')

# Виводимо перші 5 рядків датасету
trainset = data.build_full_trainset()
for i, (uid, iid, rating) in enumerate(trainset.all_ratings()):
    if i < 5:
        print(f'({trainset.to_raw_uid(uid)}, {trainset.to_raw_iid(iid)}, {rating})')
    else:
        break

# Функція для оцінки моделі
def evaluate_model(algo, data, measures=['MAE']):
    kf = KFold(n_splits=3)  # Використовуємо 3 фолди, щоб уникнути довгих обчислень
    cross_validate(algo, data, measures=measures, cv=kf, verbose=True)

# 1. Алгоритм SVD
print("\nSVD:")
svd_algo = SVD(n_factors=150, lr_all=0.01, reg_all=0.1)
evaluate_model(svd_algo, data)

# 2. Алгоритм KNNBasic
print("\nKNNBasic (Cosine Similarity):")
sim_options = {
    'name': 'cosine',
    'user_based': False  # Item-based рекомендації
}
knn_algo = KNNBasic(sim_options=sim_options)
evaluate_model(knn_algo, data)

# Пошук найкращих параметрів для SVD за допомогою GridSearchCV
param_grid = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}
gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=3)
gs.fit(data)

# Виводимо найкращі параметри для SVD
print("\nНайкращі параметри для SVD:")
print(gs.best_params['mae'])

# Виводимо рекомендації для конкретного користувача
def show_recommendations(algo, user_id, n_recommendations=10):
    trainset = data.build_full_trainset()
    algo.fit(trainset)
    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)

    recommendations = [(iid, est) for uid, iid, true_r, est, _ in predictions if uid == str(user_id)]
    recommendations.sort(key=lambda x: x[1], reverse=True)
    print(f"\nТоп {n_recommendations} фільмів для користувача {user_id}:")
    for iid, rating in recommendations[:n_recommendations]:
        print(f"Фільм ID: {iid}, Оцінка: {rating:.2f}")

# Показати рекомендації для користувача 196
show_recommendations(svd_algo, user_id=196)


(196, 242, 3.0)
(196, 393, 4.0)
(196, 381, 4.0)
(196, 251, 3.0)
(196, 655, 5.0)

SVD:
Evaluating MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     0.7345  0.7323  0.7363  0.7344  0.0016  
Fit time          2.16    1.58    1.60    1.78    0.27    
Test time         1.08    0.39    0.24    0.57    0.37    

KNNBasic (Cosine Similarity):
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     0.8247  0.8249  0.8181  0.8226  0.0031  
Fit time          0.71    0.93    0.64    0.76    0.12    
Test time         6.10    5.70    6.92    6.24    0.51    

Найкращі параметри для SVD:
{'n_factors': 150, 'lr_all': 0.01, 'reg_all': 0.1}

Т